# Modlue 2. Training with CutMix
---

본 모듈에서는 Amzaon SageMaker API 호출 없이 PyTorch 프레임워크 자체 구현만으로 모델 훈련을 수행해 봅니다. PyTorch의 문법 및 용법에 익숙하신 분들은 이 모듈을 건너 뛰고 Module 3으로 곧바로 진행하셔도 됩니다.

훈련을 원활하게 수행하시려면 GPU가 장착된 SageMaker notebook instance를 사용하셔야 합니다. (예: `g4dn.xlarge, p2.xlarge, p3.xlarge`)

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import os
import time, datetime

import warnings
warnings.filterwarnings('ignore')

from torch.utils.data import Dataset
from sklearn.metrics import recall_score

import matplotlib.pyplot as plt
import joblib

HEIGHT = 137
WIDTH = 236
BATCH_SIZE = 256

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

PyTorch 버전을 최신 버전으로 업데이트합니다. 

In [2]:
#!pip install -U torch 
#!pip install albumentations

## 1. Splits Data

Metric 평가를 위해, 훈련 데이터셋/검증 데이터셋을 분리합니다. 본 핸즈온에서는 훈련 데이터셋을 5-fold로 분리 후 4번째 fold를 검증 데이터셋으로 사용하겠습니다.

In [3]:
train_df = pd.read_csv('./input/train_folds.csv')

num_folds = 5
vld_fold_idx = 4
trn_fold = [i for i in range(num_folds) if i not in [vld_fold_idx]]
vld_fold = [vld_fold_idx]

trn_idx = train_df.loc[train_df['fold'].isin(trn_fold)].index
vld_idx = train_df.loc[train_df['fold'].isin(vld_fold)].index

## 2. Data Transformation w/ Augmentation

본 핸즈온에서는 `albumentations` 패키지를 사용하여 Data augmentation을 수행합니다. 물론 `torchvision.transforms`에서도 이미 augmentation을 지원하고 있지만, 더 많은 augmentation 기법들을 지원하고 있으며 수행 속도 또한 훨씬 빠릅니다.
관련 내용은 아래 URL을 참조해 주세요.

- [Albumentations: fast and flexible image augmentations](https://arxiv.org/pdf/1809.06839.pdf)
- [migrating_from_torchvision_to_albumentations.ipynb](https://github.com/albumentations-team/albumentations_examples/blob/master/notebooks/migrating_from_torchvision_to_albumentations.ipynb)

In [4]:
from albumentations import (
    Rotate,HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose
)
from albumentations.pytorch import ToTensor, ToTensorV2

In [5]:
train_transforms = Compose([
    Rotate(20),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),            
        ], p=0.3),
        HueSaturationValue(p=0.3),
    ToTensor()
    ], p=1.0)


valid_transforms = Compose([
    ToTensor()
])

## 3. Make DataLoader
PyTorch는 미니배치를 쉽게 로딩할 수 있는 `torch.utils.data.Dataset`과 `torch.utils.data.DataLoader`를 제공하며, 이를 사용하여
미니배치 셔플링(shuffling), 병렬 처리를 쉽게 구현할 수 있습니다.

또한, `torch.utils.data.Dataset`을 상속받아 사용자 정의 Dataset을 아래와 같이 쉽게 구성할 수 있습니다.<br>
사용자 정의 Dataset 구현의 기본적인 뼈대는 데이터셋 초기화에 필요한 `__init__(self)` 메서드, 데이터셋의 총 샘플 수를 리턴해 주는 `__len__(self)` 메서드, 특정 1개의 샘플을 가져오는 `__getitem__(self, index)` 메서드의 구현입니다.

In [6]:
class BangaliDataset(Dataset):
    def __init__(self, df, img_height, img_width, transform):
        self.df = df.reset_index()
        self.img_ids = df['image_id'].values
        self.img_height = img_height
        self.img_width = img_width
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_id = self.img_ids[index]  # e.g., Train_5
        img = joblib.load(f'./input/train_images/{img_id}.pkl')
        img = img.reshape(self.img_height,self.img_width).astype(np.uint8)
        img = 255 - img
        
        img = img[:,:,np.newaxis]
        img = np.repeat(img, 3, axis=2)
        
        if self.transform is not None:
            img = self.transform(image=img)['image']
        
        label_1 = self.df.iloc[index].grapheme_root
        label_2 = self.df.iloc[index].vowel_diacritic
        label_3 = self.df.iloc[index].consonant_diacritic
        
        return img, np.array([label_1, label_2, label_3])

In [7]:
from torch.utils.data import Dataset, DataLoader
trn_dataset = BangaliDataset(df=train_df.loc[trn_idx],
                             img_height=HEIGHT, 
                             img_width=WIDTH, 
                             transform=train_transforms)
vld_dataset = BangaliDataset(df=train_df.loc[vld_idx],
                             img_height=HEIGHT, 
                             img_width=WIDTH, 
                             transform=train_transforms)

In [8]:
trn_loader = DataLoader(trn_dataset, shuffle=True, num_workers=4, batch_size=BATCH_SIZE)
vld_loader = DataLoader(vld_dataset, shuffle=False, num_workers=4, batch_size=BATCH_SIZE)

## 4. Make model, optimizer, and criterion for Transfer Learning

모델 파라메터들을 random 분포로 초기화화여 훈련하는 것보다 이미 ImageNet 데이터로 pre-trained되어 있는 모데 파라메터를 사용하는 것이 더 효율적으로 훈련을 수행하는 방법입니다.
어떤 모델을 사용해도 무방하지만, 본 핸즈온에서는 비교적 간단한 모델인 ResNet-18을 사용하도록 하겠습니다.

In [9]:
from torchvision import datasets, models

model = models.resnet18(pretrained=True)
last_hidden_units = model.fc.in_features
model.fc = torch.nn.Linear(last_hidden_units, 186)
model = model.cuda()

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max',
                                                      verbose=True, patience=5, 
                                                      factor=0.5)

In [13]:
# CutMix Helper function
# Retrieved from https://github.com/clovaai/CutMix-PyTorch/blob/master/train.py
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    # 폭과 높이는 주어진 이미지의 폭과 높이의 beta distribution에서 뽑은 lambda로 얻는다
    cut_rat = np.sqrt(1. - lam)
    
    # patch size 의 w, h 는 original image 의 w,h 에 np.sqrt(1-lambda) 를 곱해준 값입니다.
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # patch의 중심점은 uniform하게 뽑힘
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

## 5. Training

In [14]:
from tqdm import tqdm
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

best_score = -1
log_interval = 10
training_stats = []
num_epochs = 5

for epoch_id in range(num_epochs):

    ################################################################################
    # ==> Training phase
    ################################################################################    
    trn_loss = []
    model.train()
    
    # Measure how long the training epoch takes.
    t0 = time.time()
    running_loss = 0.0
    
    for batch_id, (inputs, targets) in enumerate((trn_loader)):
        inputs = inputs.cuda()
        targets = targets.cuda()
        targets_gra = targets[:, 0]
        targets_vow = targets[:, 1]
        targets_con = targets[:, 2]
                    
        # 50%의 확률로 원본 데이터 그대로 사용    
        if np.random.rand() < 0.5:
            logits = model(inputs)
            grapheme = logits[:, :168]
            vowel = logits[:, 168:179]
            cons = logits[:, 179:]
            
            loss1 = loss_fn(grapheme, targets_gra)
            loss2 = loss_fn(vowel, targets_vow)
            loss3 = loss_fn(cons, targets_con) 
        
        else:
            
            lam = np.random.beta(1.0, 1.0) 
            rand_index = torch.randperm(inputs.size()[0])
            shuffled_targets_gra = targets_gra[rand_index]
            shuffled_targets_vow = targets_vow[rand_index]
            shuffled_targets_con = targets_con[rand_index]
            
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
            # 픽셀 비율과 정확히 일치하도록 lambda 파라메터 조정  
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
            
            logits = model(inputs)
            grapheme = logits[:,:168]
            vowel = logits[:, 168:179]
            cons = logits[:, 179:]
            
            loss1 = loss_fn(grapheme, targets_gra) * lam + loss_fn(grapheme, shuffled_targets_gra) * (1. - lam)
            loss2 = loss_fn(vowel, targets_vow) * lam + loss_fn(vowel, shuffled_targets_vow) * (1. - lam)
            loss3 = loss_fn(cons, targets_con) * lam + loss_fn(cons, shuffled_targets_con) * (1. - lam)
        
        loss = 0.5 * loss1 + 0.25 * loss2 + 0.25 * loss3    
        trn_loss.append(loss.item())
        running_loss += loss.item()
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        # Printing vital information
        if (batch_id + 1) % (log_interval) == 0:
            s = f'[Epoch {epoch_id} Batch {batch_id+1}/{len(trn_loader)}] ' \
            f'loss: {running_loss / log_interval:.4f}'
            print(s)
            running_loss = 0

        
        
    # Measure how long this epoch took.
    trn_time = format_time(time.time() - t0)        

    ################################################################################
    # ==> Validation phase
    ################################################################################
    val_loss = []
    val_true = []
    val_pred = []
    model.eval()

    with torch.no_grad():
        for inputs, targets in vld_loader:
            inputs = inputs.cuda()
            targets = targets.cuda()
            logits = model(inputs)
            grapheme = logits[:,:168]
            vowel = logits[:, 168:179]
            cons = logits[:, 179:]

            loss= 0.5* loss_fn(grapheme, targets[:,0]) + 0.25*loss_fn(vowel, targets[:,1]) + \
            0.25*loss_fn(vowel, targets[:,2])
            val_loss.append(loss.item())


            grapheme = grapheme.cpu().argmax(dim=1).data.numpy()
            vowel = vowel.cpu().argmax(dim=1).data.numpy()
            cons = cons.cpu().argmax(dim=1).data.numpy()

            val_true.append(targets.cpu().numpy())
            val_pred.append(np.stack([grapheme, vowel, cons], axis=1))                

    val_true = np.concatenate(val_true)
    val_pred = np.concatenate(val_pred)
    val_loss = np.mean(val_loss)
    trn_loss = np.mean(trn_loss)

    score_g = recall_score(val_true[:,0], val_pred[:,0], average='macro')
    score_v = recall_score(val_true[:,1], val_pred[:,1], average='macro')
    score_c = recall_score(val_true[:,2], val_pred[:,2], average='macro')
    final_score = np.average([score_g, score_v, score_c], weights=[2,1,1])
   
    # Printing vital information
    print('== Start Validation ==')
    s = f'[Epoch {epoch_id}] ' \
    f'trn_loss: {trn_loss:.4f}, vld_loss: {val_loss:.4f}, score: {final_score:.4f}, ' \
    f'score_each: [{score_g:.4f}, {score_v:.4f}, {score_c:.4f}]'          
    print(s)

    ################################################################################
    # ==> Save checkpoint and training stats
    ################################################################################        
    if final_score > best_score:
        best_score = final_score
        state_dict = model.cpu().state_dict()
        model = model.cuda()
        torch.save(state_dict, 'model.pt')
        
    # Record all statistics from this epoch
    training_stats.append(
        {
            'epoch': epoch_id + 1,
            'trn_loss': trn_loss,
            'trn_time': trn_time,            
            'val_loss': val_loss,
            'score': final_score,
            'score_g': score_g,
            'score_v': score_v,
            'score_c': score_c            
        }
    )            

[Epoch 0 Batch 10/628] loss: 3.2058
[Epoch 0 Batch 20/628] loss: 2.7928
[Epoch 0 Batch 30/628] loss: 2.5698
[Epoch 0 Batch 40/628] loss: 2.4763
[Epoch 0 Batch 50/628] loss: 2.3434
[Epoch 0 Batch 60/628] loss: 2.2434
[Epoch 0 Batch 70/628] loss: 1.7695
[Epoch 0 Batch 80/628] loss: 1.9345
[Epoch 0 Batch 90/628] loss: 2.0418
[Epoch 0 Batch 100/628] loss: 2.0295
[Epoch 0 Batch 110/628] loss: 1.7038
[Epoch 0 Batch 120/628] loss: 1.7669
[Epoch 0 Batch 130/628] loss: 1.5865
[Epoch 0 Batch 140/628] loss: 1.7775
[Epoch 0 Batch 150/628] loss: 1.7740
[Epoch 0 Batch 160/628] loss: 1.4519
[Epoch 0 Batch 170/628] loss: 1.5003
[Epoch 0 Batch 180/628] loss: 1.7282
[Epoch 0 Batch 190/628] loss: 1.5326
[Epoch 0 Batch 200/628] loss: 1.4592
[Epoch 0 Batch 210/628] loss: 1.9410
[Epoch 0 Batch 220/628] loss: 1.3942
[Epoch 0 Batch 230/628] loss: 1.6176
[Epoch 0 Batch 240/628] loss: 1.7659
[Epoch 0 Batch 250/628] loss: 1.7061
[Epoch 0 Batch 260/628] loss: 1.5860
[Epoch 0 Batch 270/628] loss: 1.4983
[Epoch 0 B

[Epoch 3 Batch 280/628] loss: 0.9319
[Epoch 3 Batch 290/628] loss: 1.2873
[Epoch 3 Batch 300/628] loss: 0.7365
[Epoch 3 Batch 310/628] loss: 1.2526
[Epoch 3 Batch 320/628] loss: 0.7849
[Epoch 3 Batch 330/628] loss: 1.0753
[Epoch 3 Batch 340/628] loss: 0.5896
[Epoch 3 Batch 350/628] loss: 1.1099
[Epoch 3 Batch 360/628] loss: 0.8727
[Epoch 3 Batch 370/628] loss: 1.3390
[Epoch 3 Batch 380/628] loss: 1.1826
[Epoch 3 Batch 390/628] loss: 0.6095
[Epoch 3 Batch 400/628] loss: 1.1146
[Epoch 3 Batch 410/628] loss: 1.8879
[Epoch 3 Batch 420/628] loss: 1.0777
[Epoch 3 Batch 430/628] loss: 0.6966
[Epoch 3 Batch 440/628] loss: 0.6807
[Epoch 3 Batch 450/628] loss: 1.2837
[Epoch 3 Batch 460/628] loss: 0.8716
[Epoch 3 Batch 470/628] loss: 0.9771
[Epoch 3 Batch 480/628] loss: 0.5411
[Epoch 3 Batch 490/628] loss: 0.8564
[Epoch 3 Batch 500/628] loss: 1.0098
[Epoch 3 Batch 510/628] loss: 1.1918
[Epoch 3 Batch 520/628] loss: 0.8603
[Epoch 3 Batch 530/628] loss: 1.2427
[Epoch 3 Batch 540/628] loss: 1.2388
[

## 6. Check training results

In [15]:
import pandas as pd
pd.set_option('precision', 4)

df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')
display(df_stats)

#model.load_state_dict(torch.load('./model.pt'))

,score,score_c,score_g,score_v,trn_loss,trn_time,val_loss
epoch,,,,,,,
1,0.8163,0.7981,0.7921,0.8829,1.5630,0:04:43,1.4987
2,0.8587,0.8546,0.8274,0.9255,1.2241,0:04:45,1.3153
3,0.8915,0.8876,0.8701,0.9382,1.0880,0:04:37,1.1834
4,0.9209,0.9395,0.8961,0.9521,1.0103,0:04:41,1.1960
5,0.9199,0.9285,0.9026,0.9460,0.9635,0:04:40,1.1583
